# Advanced Querying Mongo

Importing libraries and setting up connection

In [3]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost/companies")

In [9]:
db = client.get_database()
c = "companies"
db[c].find()[:1]

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [23]:
# Your Code
babel = list(db[c].find({"name":"Babelgum"}))
babel[0]
len(babel)
babel[0].keys()
babel[0]['name']

'Babelgum'

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [62]:
bigcompanies = db[c].find({"number_of_employees": {"$gt": 5000}}).limit(20).sort("number_of_employees", -1)
for dict in bigcompanies:
    print(dict['name'], dict['number_of_employees'])

Siemens 405000
IBM 388000
Toyota 320000
PayPal 300000
Nippon Telegraph and Telephone Corporation 227000
Samsung Electronics 221726
Accenture 205000
Tata Consultancy Services 200300
Flextronics International 200000
Safeway 186000
Sony 180500
LG 177000
Ford 171000
Boeing 160000
Digital Equipment Corporation 140000
Nokia 125000
MItsubishi Electric 107000
MItsubishi Electric 107000
Comcast 100000
Bertelsmann 100000


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [67]:
comps2k = db[c].find({"$and": [{"founded_year": {"$gte":2000}}, {"founded_year":{"$lte":2005}}]})
for e in comps2k[:20]:
    print(e["name"],' ',e['founded_year'])


Wetpaint   2005
Omnidrive   2005
Zoho   2005
Gizmoz   2003
Digg   2004
Helio   2005
StumbleUpon   2002
Plaxo   2002
Technorati   2002
AddThis   2004
Veoh   2004
Jingle Networks   2005
Meetup   2002
Facebook   2004
LifeLock   2005
Jangl SMS   2005
Wesabe   2005
SmugMug   2002
Jajah   2005
YouTube   2005


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [114]:
# def getattrib(cursor,*args, lim=10):
#     return [[e[arg] for e in cursor[:lim]] for arg in args]

# def getattrib(cursor,*args, lim=10):
#     for e in cursor[:lim]:
#         print(";".join([e[arg] for arg in args]))

def names(cursor, lim=10):
    for e in cursor[:lim]:
        print(e['name'])

In [82]:
valuable = db[c].find({"$and": [ {"ipo.valuation_amount":{"$gte":1E8}}, {"founded_year": {"$lt": 2010}} ] })


for e in valuable[:5]:
    print(e["name"], e["ipo"])


Twitter {'valuation_amount': 18100000000, 'valuation_currency_code': 'USD', 'pub_year': 2013, 'pub_month': 11, 'pub_day': 7, 'stock_symbol': 'NYSE:TWTR'}
Facebook {'valuation_amount': 104000000000, 'valuation_currency_code': 'USD', 'pub_year': 2012, 'pub_month': 5, 'pub_day': 18, 'stock_symbol': 'NASDAQ:FB'}
Yelp {'valuation_amount': 1300000000, 'valuation_currency_code': 'USD', 'pub_year': 2012, 'pub_month': 3, 'pub_day': 2, 'stock_symbol': 'NYSE:YELP'}
LinkedIn {'valuation_amount': 9310000000, 'valuation_currency_code': 'USD', 'pub_year': 2011, 'pub_month': 7, 'pub_day': 20, 'stock_symbol': 'NYSE:LNKD'}
Brightcove {'valuation_amount': 290000000, 'valuation_currency_code': 'USD', 'pub_year': 2012, 'pub_month': 2, 'pub_day': 17, 'stock_symbol': 'NASDAQ:BCOV'}


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [90]:
smol = db[c].find({"$and": [ {"number_of_employees": {"$lt": 1000}}, {"founded_year": {"$lt": 2005}} ] } ).limit(10)

for e in smol:
    print(e["name"])

AdventNet
Digg
Fox Interactive Media
Plaxo
Technorati
AddThis
Meetup
SmugMug
Pando Networks
Ikan


### 6. All the companies that don't include the `partners` field.

In [138]:
nopartner = db[c].find({"partners": "null" })

#nopartner[0] 
#Returns an error, bc the query is empty, there no companies without a partners field.

In [140]:
nopartner = db[c].find({"partners": []}) #Asi si los encuentra  

names(nopartner)

AdventNet
Wetpaint
Omnidrive
Zoho
Postini
Geni
Gizmoz
Digg
Slacker
Scribd


### 7. All the companies that have a null type of value on the `category_code` field.

In [112]:
nocat = db[c].find( { "category_code" : { "$type": 10 } })
#for e in nocat['name']

names(nocat)

Collective
KoolIM
Snimmer
Level9 Media
VidKing
Drigg
SpaceTime
Touch Clarity
MMDAYS
Inside Group


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [117]:
medium = db[c].find({"$and": [{"number_of_employees":{"$gte":100}}, {"number_of_employees":{"$lt":1000}}]})
for e in medium[:10]:
    print(e['name'],e["number_of_employees"])

AdventNet 600
AddThis 120
LifeLock 644
OpenX 305
Jajah 110
Livestream 120
Ustream 250
iContact 300
Yelp 800
Dailymotion 120


### 9. Order all the companies by their IPO price in a descending order.

In [122]:
orderprice = db[c].find().sort("ipo.valuation_amount", -1)
names(orderprice)

GREE
Facebook
Amazon
Twitter
Groupon
Tencent
Western Digital
LinkedIn
BMC Software
Rackspace


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [124]:
comp = db[c]
emp10 = comp.find().sort("number_of_employees", -1).limit(10)
names(emp10)

Siemens
IBM
Toyota
PayPal
Nippon Telegraph and Telephone Corporation
Samsung Electronics
Accenture
Tata Consultancy Services
Flextronics International
Safeway


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [175]:
rang = [e for e in range(6,13)]
secondtrim = comp.find({"founded_month":{"$in": rang}}).limit(1000)
names(secondtrim)

Wetpaint
Omnidrive
Zoho
Postini
Geni
Digg
Fox Interactive Media
Joost
Plaxo
Technorati


### 12. All the companies founded before 2000 that have an acquisition amount of more than 1.000.000

In [142]:
bef2000 = comp.find({"$and":[{"founded_year":{"$lt":2000}},{"acquisition.price_amount":{"$gt":1E6}}]})
names(bef2000)

Postini
SideStep
Recipezaar
Cyworld
PayPal
Snapfish
Neopets
Zappos
Alibaba
Sun Microsystems


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [149]:
after2010 = comp.find({"acquisition.acquired_year":{"$gt":2010}}).sort("acquisition.price_amount")

for e in after2010[:5]:
    print(e["name"], e["acquisition"])

Geni {'price_amount': None, 'price_currency_code': 'USD', 'term_code': None, 'source_url': 'http://techcrunch.com/2012/11/28/all-in-the-family-myheritage-buys-former-yammer-stablemate-geni-com-raises-25m/', 'source_description': 'MyHeritage acquires Geni and $25M to build family tree of the whole world', 'acquired_year': 2012, 'acquired_month': 11, 'acquired_day': 28, 'acquiring_company': {'name': 'MyHeritage', 'permalink': 'myheritage'}}
Kyte {'price_amount': None, 'price_currency_code': 'USD', 'term_code': None, 'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/', 'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million', 'acquired_year': 2011, 'acquired_month': 1, 'acquired_day': 31, 'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}
blogTV {'price_amount': None, 'price_currency_code': 'USD', 'term_code': None, 'source_url': 'http://techcrunch.com/2013/03/13/y

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [160]:
founded = comp.find({"founded_year":{"$ne":None}}).sort("founded_year",1)
for e in founded[:10]:
    print(e['name'], e['founded_year'])

Alstrasoft 1800
SmallWorlds 1800
US Army 1800
DuPont 1802
McKesson 1833
Bachmann Industries 1833
Bertelsmann 1835
Accuity 1836
CENTRA 1839
WeGame 1840


### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [170]:
rang = [e for e in range(1,8)]
lucky7 = comp.find({"founded_day":{"$in":rang}}).sort("acquisition price",-1)
names(lucky7)

Omnidrive
Postini
Geni
Gizmoz
Fox Interactive Media
Helio
Joost
StumbleUpon
Babelgum
Technorati


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [172]:
webs = comp.find({"$and": [{"category_code":"web"}, {"number_of_employees":{"$gt":4000}}]}).sort("number_of_employees",1)
names(webs)



Expedia
AOL
Webkinz
Rakuten
Los Angeles Times Media Group
Groupon
Yahoo!
eBay
Experian


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [174]:
expensiveEUR = comp.find({"$and":[{"acquisition.price_amount":{"$gt":1E4}}, {"acquisition.price_currency_code":"EUR"}]})
names(expensiveEUR)

ZYB
Apertio
Greenfield Online
Mobile Trend
Citizenside
Excite Europe
YelloYello
Webedia
Wayfinder
Branded Payment Solutions


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [178]:
rang = [e for e in range(1,4)]
firsttrim = comp.find({"acquisition.acquired_month":{"$in": rang}}).limit(10)
for e in firsttrim[:5]:
    print(e["name"], e["acquisition"])


Kyte {'price_amount': None, 'price_currency_code': 'USD', 'term_code': None, 'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/', 'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million', 'acquired_year': 2011, 'acquired_month': 1, 'acquired_day': 31, 'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}
NetRatings {'price_amount': 327000000, 'price_currency_code': 'USD', 'term_code': 'cash', 'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L', 'source_description': 'Nielsen buys rest of NetRatings', 'acquired_year': 2007, 'acquired_month': 2, 'acquired_day': None, 'acquiring_company': {'name': 'Nielsen', 'permalink': 'nielsen'}}
blogTV {'price_amou

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code